<a href="https://colab.research.google.com/github/FoaadAbbas/B.sc/blob/main/trgol2CloudComputing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# שלב 0: התקנת הספריות הנדרשות
!pip install -q google-api-python-client

# --- שלב 1: ייבוא ספריות ---
import json
import os
import io
import ipywidgets as widgets
from IPython.display import display, HTML

# ייבואים ספציפיים לגישה ל-Google Drive API
from google.colab import auth
from google.auth import default
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload, MediaIoBaseDownload

# --- שלב 2: הגדרות קבועות ---
# זהו ה-ID שהוצא מהקישור שסיפקת
FILE_ID = '1qQpdJhXlgK55PCt2B8xY0wgpEoY56m3P'
# שם הקובץ הזמני בסביבת הריצה של Colab
LOCAL_FILE_PATH = '/content/students.json'

# --- שלב 3: פונקציות עזר (קריאה וכתיבה מ-Drive) ---

def load_data_from_drive(drive_service, file_id, local_path):
    """
    מוריד קובץ מ-Google Drive באמצעות ה-API
    וטוען אותו כאובייקט JSON.
    """
    try:
        request = drive_service.files().get_media(fileId=file_id)
        fh = io.FileIO(local_path, 'wb')
        downloader = MediaIoBaseDownload(fh, request)

        done = False
        while done is False:
            status, done = downloader.next_chunk()
            print(f"הורדה... {int(status.progress() * 100)}%.")

        print(f"קובץ '{local_path}' הורד בהצלחה.")

        with open(local_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
            return data

    except Exception as e:
        print(f"שגיאה חמורה בטעינת הקובץ: {e}")
        return None

def save_data_to_drive(drive_service, file_id, local_path, data):
    """
    מעדכן את הקובץ הקיים ב-Google Drive עם הנתונים החדשים.
    """
    try:
        with open(local_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        media_body = MediaFileUpload(local_path, mimetype='application/json', resumable=True)

        updated_file = drive_service.files().update(
            fileId=file_id,
            media_body=media_body
        ).execute()

        print(f"\n✅ עדכון נשמר בהצלחה ב-Google Drive!")
        return True

    except Exception as e:
        print(f"שגיאה בשמירת הקובץ: {e}")
        return False

# --- שלב 4: עיצוב הטופס (CSS) ---
display(HTML("""
<style>
    .student-form {
        background-color: #ffffff;  /* <--- שינוי: רקע לבן */
        border: 1px solid #cccccc;
        border-radius: 12px;
        padding: 24px;
        box-shadow: 0 4px 10px rgba(0,0,0,0.05);
        font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    }
    .student-form .widgets-label {
        font-weight: bold;
        color: #191970;
        margin-bottom: 5px;
    }
    /* עיצוב שדות הניתנים לעריכה */
    .student-form .widget-text input[type="text"],
    .student-form .widget-textarea textarea {
        background-color: #f9f9f9; /* <--- שינוי: רקע אפרפר בהיר לשדות עריכה */
        border: 1px solid #ccc;
        border-radius: 5px;
        width: 95%;
    }
    .student-form .widget-dropdown select {
        font-size: 1.1em;
        width: 100%;
        background-color: white;
    }
    .student-form .widget-button button {
        background-color: #007bff; /* צבע כחול-עדכון */
        color: white;
        font-weight: bold;
        border: none;
        border-radius: 5px;
        padding: 10px 15px;
        cursor: pointer;
        transition: background-color 0.2s;
    }
    .student-form .widget-button button:hover {
        background-color: #0056b3;
    }
</style>
"""))

# --- שלב 5: הרצת הלוגיקה הראשית ---
try:
    print("אנא אשר גישה ל-Google Drive...")
    auth.authenticate_user()
    creds, _ = default()
    drive_service = build('drive', 'v3', credentials=creds)
    print("אימות עבר בהצלחה. מתחיל בטעינת הנתונים...")

    students_data = load_data_from_drive(drive_service, FILE_ID, LOCAL_FILE_PATH)

    if students_data:
        student_names = [f"{s.get('שם_פרטי', '')} {s.get('שם_משפחה', '')}" for s in students_data]

        # --- יצירת רכיבי הטופס ---
        student_dropdown = widgets.Dropdown(options=student_names, description='בחר סטודנט:')

        # --- שינוי: כל השדות כעת ניתנים לעריכה (disabled=False) ---
        first_name_text = widgets.Text(description='שם פרטי:', disabled=False)
        last_name_text = widgets.Text(description='שם משפחה:', disabled=False)
        email_text = widgets.Text(description='מייל:', disabled=False, layout=widgets.Layout(width='95%'))
        # הערה: קורסים יופרדו בפסיק
        courses_text = widgets.Textarea(description='קורסים (מופרד בפסיק):', disabled=False, layout=widgets.Layout(height='80px', width='95%'))
        link_text = widgets.Text(description='קישור:', disabled=False, layout=widgets.Layout(width='95%'))
        favorite_show_text = widgets.Text(description='תוכנית אהובה:', placeholder='הזן תוכנית אהובה...', disabled=False, layout=widgets.Layout(width='95%'))

        update_button = widgets.Button(description='שמור שינויים')
        output_area = widgets.Output()

        # --- פונקציות אירועים ---

        def on_student_change(change):
            """מעדכן את כל התיבות כאשר בוחרים סטודנט חדש"""
            selected_name = change['new']
            student = next((s for s in students_data if f"{s.get('שם_פרטי', '')} {s.get('שם_משפחה', '')}" == selected_name), None)

            if student:
                first_name_text.value = student.get('שם_פרטי', '')
                last_name_text.value = student.get('שם_משפחה', '')
                email_text.value = student.get('מייל_בראודה', '')
                # הופך את רשימת הקורסים למחרוזת מופרדת בפסיקים
                courses_text.value = ', '.join(student.get('קורסים', []))
                link_text.value = student.get('קישור_מענין', '')
                favorite_show_text.value = student.get('תוכנית_אהובה', '')

        def on_update_button_click(b):
            """
            --- שינוי: קורא את כל הנתונים מהטופס ומעדכן את הקובץ ---
            """
            with output_area:
                output_area.clear_output()
                selected_name_on_load = student_dropdown.value

                # איתור הסטודנט ברשימת הנתונים המקורית
                student_index = next((i for i, s in enumerate(students_data)
                                      if f"{s.get('שם_פרטי', '')} {s.get('שם_משפחה', '')}" == selected_name_on_load), -1)

                if student_index != -1:
                    print("מעדכן נתונים...")

                    # קריאת כל הערכים מהטופס
                    students_data[student_index]['שם_פרטי'] = first_name_text.value
                    students_data[student_index]['שם_משפחה'] = last_name_text.value
                    students_data[student_index]['מייל_בראודה'] = email_text.value
                    students_data[student_index]['קישור_מענין'] = link_text.value
                    students_data[student_index]['תוכנית_אהובה'] = favorite_show_text.value

                    # הפיכת המחרוזת בחזרה לרשימה
                    courses_list = [course.strip() for course in courses_text.value.split(',') if course.strip()]
                    students_data[student_index]['קורסים'] = courses_list

                    # שמירת הנתונים המעודכנים בחזרה לקובץ ב-Drive
                    success = save_data_to_drive(drive_service, FILE_ID, LOCAL_FILE_PATH, students_data)

                    if success:
                        # רענון הרשימה הנפתחת אם השם שונה
                        new_full_name = f"{first_name_text.value} {last_name_text.value}"
                        global student_names
                        student_names = [f"{s.get('שם_פרטי', '')} {s.get('שם_משפחה', '')}" for s in students_data]

                        # עדכון האפשרויות ברשימה הנפתחת
                        student_dropdown.options = student_names
                        # בחירה מחדש של הסטודנט (עם השם החדש, אם שונה)
                        student_dropdown.value = new_full_name
                else:
                    print("שגיאה: לא נמצא סטודנט לעדכון.")

        # --- קישור הפונקציות לרכיבים ---
        student_dropdown.observe(on_student_change, names='value')
        update_button.on_click(on_update_button_click)

        # --- הצגת הטופס ---
        form_layout = widgets.VBox([
            student_dropdown,
            first_name_text,
            last_name_text,
            email_text,
            courses_text,
            link_text,
            widgets.HTML("<hr style='border-color: #ccc;'>"), # קו מפריד
            favorite_show_text,
            update_button,
            output_area
        ])

        form_layout.add_class('student-form') # הוספת הקלאס העיצובי

        # הפעלה ראשונית כדי למלא נתונים
        if student_names:
            on_student_change({'new': student_names[0]})

        display(form_layout)

    else:
        print("לא נטענו נתונים. בדוק את הרשאות הקובץ או את תקינותו.")

except Exception as e:
    print(f"שגיאה כללית בהרצת הקוד: {e}")
    print("--- ייתכן שלא אישרת את הגישה ל-Drive. נסה להריץ את התא מחדש. ---")

אנא אשר גישה ל-Google Drive...
אימות עבר בהצלחה. מתחיל בטעינת הנתונים...
הורדה... 100%.
קובץ '/content/students.json' הורד בהצלחה.
